Here we will make our labelmaps more realistic by superimposing hippocampal segmentations onto labelmaps from fastsurfer/synthseg

We do this for two cases: one with a deep collateral sulcus one one with a split. We also flip our samples so they are each fit onto a left and a right background. Finally, in on case, we also leave the backgroud blank.

In [1]:
import nibabel as nib
import numpy as np
from scipy.ndimage import binary_dilation, label

In [2]:
import numpy as np
from scipy.ndimage import label, find_objects

def largest_connected_component(binary_mask):
    """
    Keeps only the largest connected component in a 3D binary mask.

    Parameters:
    - binary_mask (numpy.ndarray): A 3D binary mask (0s and 1s).

    Returns:
    - numpy.ndarray: A 3D binary mask with only the largest connected component.
    """
    labeled_array, num_features = label(binary_mask)  # Label connected components
    if num_features == 0:
        return binary_mask  # No components, return original

    # Count voxels in each component
    component_sizes = np.bincount(labeled_array.ravel())
    component_sizes[0] = 0  # Ignore background (label 0)
    
    # Get largest component label
    largest_label = component_sizes.argmax()

    # Create mask of largest component
    return (labeled_array == largest_label).astype(binary_mask.dtype)


In [3]:
subs = ['PNC001', 'PNC004'] # one split collateral sulcus, one deep collateral sulcus

In [6]:
!mkdir bgs
for sub in subs:
    aseg = nib.load(f"/data/mica3/BIDS_PNI/derivatives/fastsurfer/sub-{sub}_ses-01/mri/aparc.DKTatlas+aseg.mgz")
    T1 = nib.load(f"/data/mica3/BIDS_PNI/derivatives/fastsurfer/sub-{sub}_ses-01/mri/orig.mgz").get_fdata()
    dat = aseg.get_fdata()

    # fill gaps between hipp and neocortex
    hippfill = np.logical_and(binary_dilation(dat==17, iterations=3), binary_dilation(dat>1000, iterations=3))
    hippfill[dat==2] = 0
    hippfill = largest_connected_component(hippfill)
    dat[hippfill==1] = 17
    hippfill = np.logical_and(binary_dilation(dat==53, iterations=3), binary_dilation(dat>1000, iterations=3))
    hippfill[dat==41] = 0
    hippfill = largest_connected_component(hippfill)
    dat[hippfill==1] = 53

    # make sure there is wm between hipp and amygdala
    wmfill = np.logical_and(binary_dilation(dat==53, iterations=2), dat==54)
    dat[wmfill==1] = 41
    wmfill = np.logical_and(binary_dilation(dat==17, iterations=2), dat==18)
    dat[wmfill==1] = 2

    # simplify some labels
    dat[T1<50] = 0 #CSF
    dat[T1>240] = 101 #blood

    nib.save(nib.Nifti1Image(dat.astype(int),header=aseg.header,affine=aseg.affine), f'bgs/sub-{sub}_ses-01_desc-aparcaseg_preproc.nii.gz')